In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
wildfire_file_name = r'D:\yeshiva\2 semester\math\final\data\California_Fire_Incidents.csv'

wildfire_df =pd.read_csv(wildfire_file_name, converters={
                               'Counties': str})

wildfire_df.shape

(1636, 40)

In [4]:
wildfire_df.head(2)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06T18:30:00Z,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,19,NaN,NaN,NaN,2013-06-08T18:30:00Z,NaN,False,True,NaN,NaN,NaN,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100.0,NaN,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN


In [5]:
wildfire_df.dtypes

AcresBurned             float64
Active                     bool
AdminUnit                object
AirTankers              float64
ArchiveYear               int64
CalFireIncident            bool
CanonicalUrl             object
ConditionStatement       object
ControlStatement         object
Counties                 object
CountyIds                object
CrewsInvolved           float64
Dozers                  float64
Engines                 float64
Extinguished             object
Fatalities              float64
Featured                   bool
Final                      bool
FuelType                 object
Helicopters             float64
Injuries                float64
Latitude                float64
Location                 object
Longitude               float64
MajorIncident              bool
Name                     object
PercentContained        float64
PersonnelInvolved       float64
Public                     bool
SearchDescription        object
SearchKeywords           object
Started 

In [6]:
#s2 = s1.astype('int64', copy=False)
#wildfire_df['Counties'] = wildfire_df['Counties'].astype('str') 
#wildfire_df.dtypes

wildfire_df = wildfire_df.convert_dtypes()
wildfire_df.dtypes

AcresBurned               Int64
Active                  boolean
AdminUnit                string
AirTankers                Int64
ArchiveYear               Int64
CalFireIncident         boolean
CanonicalUrl             string
ConditionStatement       string
ControlStatement         string
Counties                 string
CountyIds                string
CrewsInvolved             Int64
Dozers                    Int64
Engines                   Int64
Extinguished             string
Fatalities                Int64
Featured                boolean
Final                   boolean
FuelType                 string
Helicopters               Int64
Injuries                  Int64
Latitude                float64
Location                 string
Longitude               float64
MajorIncident           boolean
Name                     string
PercentContained          Int64
PersonnelInvolved         Int64
Public                  boolean
SearchDescription        string
SearchKeywords           string
Started 

# 1) Add median income level

In [7]:
income_file_name = r'D:\yeshiva\2 semester\math\final\data\California_Median_Income.csv'

income_df =pd.read_csv(income_file_name)

income_df.shape

(58, 2)

In [8]:
income_df.head(2)

,County,Value
0,Alameda,92574.0
1,Alpine,64688.0


In [9]:
income_dict = {}

for ind, row in income_df.iterrows():
    income_dict[row['County']] = row['Value']

In [10]:
wildfire_df['median_income'] = wildfire_df.apply(lambda row: income_dict.get(row['Counties']), axis=1)

In [11]:
wildfire_df['median_income']

0       56493.0
1       64251.0
2       63948.0
3       84357.0
4       84017.0
         ...   
1631    63948.0
1632    63240.0
1633    65923.0
1634    74855.0
1635    63948.0
Name: median_income, Length: 1636, dtype: float64

In [12]:
median_income_quantiles = {}
median_income_quantiles['q1'] = wildfire_df['median_income'].quantile(.25)
median_income_quantiles['q2'] = wildfire_df['median_income'].quantile(.50)
median_income_quantiles['q3'] = wildfire_df['median_income'].quantile(.75)

In [13]:
def income_level(a):
    level = 1
    if a <= median_income_quantiles.get('q1'):
        level = 1
    elif a > median_income_quantiles.get('q1') and a <= median_income_quantiles.get('q2'):
        level = 2
    elif a > median_income_quantiles.get('q2') and a <= median_income_quantiles.get('q3'):
        level = 3
    else:
        level = 4
        
    return level

In [14]:
wildfire_df['median_income_level'] = wildfire_df.apply(lambda row: income_level(row['median_income']), axis=1)
wildfire_df['median_income_level']

0       2
1       3
2       3
3       4
4       4
       ..
1631    3
1632    3
1633    3
1634    4
1635    3
Name: median_income_level, Length: 1636, dtype: int64

In [15]:
wildfire_df.to_csv(r'D:\yeshiva\2 semester\math\final\data\California_Fire_Incidents_with_income.csv', index=False)

In [16]:
wildfire_df.head(2)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders,median_income,median_income_level
0,257314,False,Stanislaus National Forest/Yosemite National Park,<NA>,2013,True,/incidents/2013/8/17/rim-fire/,<NA>,<NA>,Tuolumne,55,<NA>,<NA>,<NA>,2013-09-06T18:30:00Z,<NA>,False,True,<NA>,<NA>,<NA>,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100,<NA>,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,<NA>,<NA>,<NA>,<NA>,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,<NA>,56493.0,2
1,30274,False,USFS Angeles National Forest/Los Angeles Count...,<NA>,2013,True,/incidents/2013/5/30/powerhouse-fire/,<NA>,<NA>,Los Angeles,19,<NA>,<NA>,<NA>,2013-06-08T18:30:00Z,<NA>,False,True,<NA>,<NA>,<NA>,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100,<NA>,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,<NA>,<NA>,<NA>,<NA>,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,<NA>,64251.0,3


# 2) Add seaside and inland ranges

In [17]:
seaside_inland_file_name = r'D:\yeshiva\2 semester\math\final\data\California_counties_of_selected_areaes.csv'

seaside_df =pd.read_csv(seaside_inland_file_name)

print(seaside_df.shape)

seaside_df.head(2)

(58, 3)


,economic_region,county,range
0,Northern California,Del Norte,seaside
1,Northern California,Siskiyou,inland


In [18]:
seaside_dict = {}

for ind, row in seaside_df.iterrows():
    seaside_dict[row['county']] = row['range']

In [19]:
wildfire_df['seaside_inland'] = wildfire_df.apply(lambda row: seaside_dict.get(row['Counties']), axis=1)
wildfire_df['seaside_inland'] 

0        inland
1       seaside
2        inland
3        inland
4       seaside
         ...   
1631     inland
1632     inland
1633     inland
1634    seaside
1635     inland
Name: seaside_inland, Length: 1636, dtype: object

In [20]:
seaside_bar_df = wildfire_df.groupby('seaside_inland').size()
seaside_bar_df

seaside_inland
inland     1067
seaside     565
dtype: int64

In [21]:
seaside_bar_df = wildfire_df[['AcresBurned', 'seaside_inland']].groupby('seaside_inland').sum()
seaside_bar_df

,AcresBurned
seaside_inland,
inland,5019786
seaside,2470778


In [22]:
seaside_bar_df = wildfire_df[['Name', 'seaside_inland']].groupby('seaside_inland').size()
seaside_bar_df

seaside_inland
inland     1067
seaside     565
dtype: int64

In [23]:
wildfire_df.to_csv(r'D:\yeshiva\2 semester\math\final\data\California_Fire_Incidents_with_income_selected_areas.csv', index=False)

# 3) include the daily climate of San Diego and Riverside 

- 3.1) select the nearest climate station for each row
- 3.2）select the climate information for each row from the nearest climate station
- 3.3) convert number to category for logical regression

In [24]:
import math
# Tel Aviv 32.0853° N, 34.7818° E
# Chennal 13.0827° N, 80.2707° E
# earth's radis 6378.137 km


def deg_rad(deg):
    return deg * math.pi / 180


def distance_km(lat1, lng1, lat2, lng2):
    earth_radis = 6378.137  # earth's radis 6378.137 km

    rad_lat1 = deg_rad(lat1)
    rad_lat2 = deg_rad(lat2)
    rad_lng1 = deg_rad(lng1)
    rad_lng2 = deg_rad(lng2)

    lat_diff = rad_lat1 - rad_lat2
    lng_diff = rad_lng1 - rad_lng2

    s = 2 * math.asin(math.sqrt(
        pow(math.sin(lat_diff / 2), 2) + math.cos(rad_lat1) * math.cos(rad_lat2) * pow(math.sin(lng_diff / 2),
                                                                                       2))) * earth_radis
    # print("distance is = %.4f km" % s)
    
    return s


distance_km(32.0853, 34.7818, 13.0827, 80.2707)

5083.9775948959195

### 3.1) select the nearest climate station for each row

In [25]:
climate_stations_file_name = r'D:\yeshiva\2 semester\math\final\data\California_SanDiego_Riverside_stations.csv'

station_df =pd.read_csv(climate_stations_file_name)

print(station_df.shape)

station_df.head(2)

(9, 4)


,Counties,stations,station_latitude,station_longitude
0,San Diego,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944
1,San Diego,"SAN DIEGO INTERNATIONAL AIRPORT, CA US",32.73360,-117.18310


In [26]:
sandiego_riverside_df = wildfire_df[(wildfire_df['Counties'] == 'San Diego') | (wildfire_df['Counties'] == 'Riverside')]

#wildfire_df['climate_station'] = wildfire_df.apply(lambda row: seaside_dict.get(row['Counties']), axis=1)

In [27]:
sandiego_riverside_df.shape

(235, 43)

In [28]:
def cal_nearest_station(wildfire_row, station_df):
    wildfire_latitude = wildfire_row['Latitude']
    wildfire_longitude = wildfire_row['Longitude']
    #print(wildfire_latitude)
    #print(wildfire_longitude)
    
    min_i = 0
    min_distance = 10000000 # 1M km
    for i, r in station_df.iterrows():
        s_latitude = r['station_latitude']
        s_longitude = r['station_longitude']
        
        distance = distance_km(wildfire_latitude, wildfire_longitude, s_latitude, s_longitude)
        if distance < min_distance:
            min_distance = distance
            min_i = i
    min_station = station_df.loc[min_i]
    
    #print(type(min_station))
    #print(len(min_station[1:]))
    return min_station[1], min_station[2], min_station[3]

In [29]:
# for ind, row in station_df.iterrows():
def select_nearest_station(wr, s_df = station_df):
    #for wi, wr in sandiego_riverside_df.iterrows():

    return cal_nearest_station(wr, s_df)
    
    #break
    

In [30]:
#wildfire_df['tmp'] = wildfire_df.apply(select_nearest_station, axis=1)

#wildfire_df[['stations', 'station_latitude', 'station_longitude']] = wildfire_df['tmp'].apply(pd.Series)
# df_tmp[["fomat1", "format2"]] = df_tmp.apply(formatrow, axis=1, result_type="expand")
sandiego_riverside_df[['stations', 'station_latitude', 'station_longitude']] = sandiego_riverside_df.apply(lambda row: select_nearest_station(row, station_df), axis=1, result_type="expand")

D:\dev\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [31]:
sandiego_riverside_df.head(2)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders,median_income,median_income_level,seaside_inland,stations,station_latitude,station_longitude
2,27531,False,CAL FIRE Riverside Unit / San Bernardino Natio...,<NA>,2013,True,/incidents/2013/7/15/mountain-fire/,<NA>,<NA>,Riverside,33,<NA>,<NA>,<NA>,2013-07-30T18:00:00Z,<NA>,False,True,<NA>,<NA>,<NA>,33.70950,Hwy 243 & Hwy 74 near Mountain Center,-116.72885,False,Mountain Fire,100,<NA>,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,<NA>,<NA>,<NA>,<NA>,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,<NA>,63948.0,3,inland,"PALM SPRINGS ASOS, CA US",33.8222,-116.5043
6,20292,False,CAL FIRE Riverside Unit / San Bernardino Natio...,<NA>,2013,True,/incidents/2013/8/7/silver-fire/,Firefighters closed the containment lines toda...,Hwy 243 remains closed between Twin Pines Road...,Riverside,33,63,20,201,2013-08-12T18:00:00Z,<NA>,False,True,<NA>,20,26,33.86157,"Poppet Flats Rd near Hwy 243, south of Banning",-116.90427,True,Silver Fire,100,2106,True,The Silver Fire burned in August 2013 off Popp...,"Silver Fire, August 2013, Popet Flats Road, Hi...",2013-08-07T14:05:00Z,Finalized,8,40,<NA>,<NA>,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,2013-08-12T18:00:00Z,20,63948.0,3,inland,"PALM SPRINGS ASOS, CA US",33.8222,-116.5043


### 3.2）select the climate information for each row from the nearest climate station

In [32]:
climate_daily_file_name = r'D:\yeshiva\2 semester\math\final\data\California_SanDiego_Riverside_climate_2017-2019.csv'

climate_daily_df =pd.read_csv(climate_daily_file_name)

print(climate_daily_df.shape)

climate_daily_df.head(2)

(9847, 23)


,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County
0,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-01-01,4.92,0.00,57.0,44.0,210.0,210.0,15.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
1,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-01-02,1.79,0.03,58.0,50.0,250.0,250.0,8.1,10.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego


In [33]:
sandiego_riverside_df['Started'].astype

<bound method NDFrame.astype of 2       2013-07-15T13:43:00Z
6       2013-08-07T14:05:00Z
11      2013-07-06T12:55:00Z
16      2013-05-01T12:38:00Z
18      2013-05-23T12:20:00Z
                ...         
1624    2019-10-15T17:41:57Z
1627    2019-09-10T10:43:58Z
1631    2019-10-10T12:08:00Z
1634    2019-10-22T19:20:44Z
1635    2019-10-14T15:32:20Z
Name: Started, Length: 235, dtype: string>

In [34]:
# dates.apply(lambda x: x.strftime('%Y-%m-%d'))
sandiego_riverside_df['started_date_str'] = sandiego_riverside_df.apply(lambda x: x['Started'][:10], axis=1)

<ipython-input-34-2689b0b5b369>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sandiego_riverside_df['started_date_str'] = sandiego_riverside_df.apply(lambda x: x['Started'][:10], axis=1)


In [35]:
sandiego_riverside_df['started_date_str'].head(2)

2    2013-07-15
6    2013-08-07
Name: started_date_str, dtype: object

In [36]:
sandiego_riverside_df = sandiego_riverside_df[sandiego_riverside_df['started_date_str'] >= '2017-01-01']
sandiego_riverside_df.shape

(164, 47)

In [37]:
climate_daily_df['DATE']

0       2017-01-01
1       2017-01-02
2       2017-01-03
3       2017-01-04
4       2017-01-05
           ...    
9842    2019-12-27
9843    2019-12-28
9844    2019-12-29
9845    2019-12-30
9846    2019-12-31
Name: DATE, Length: 9847, dtype: object

In [38]:
sandiego_riverside_df['started_date_str']

652     2017-06-26
661     2017-12-07
663     2017-09-02
673     2017-05-20
688     2017-04-30
           ...    
1624    2019-10-15
1627    2019-09-10
1631    2019-10-10
1634    2019-10-22
1635    2019-10-14
Name: started_date_str, Length: 164, dtype: object

In [39]:
lst = []
for wi, wr in sandiego_riverside_df.iterrows():
    # get the fire if it mach the station and the date
    lst.append(climate_daily_df[(climate_daily_df['NAME'] == wr['stations']) & (climate_daily_df['DATE'] == wr['started_date_str'])])

lst
fire_nearest_climate_df = pd.concat(lst)

climate_dic = {}
for ni, nr in fire_nearest_climate_df.iterrows():
    key = nr['NAME'] + nr['DATE']
    # print(key)
    climate_dic[key] = nr.to_numpy()

# print(type(climate_dic.get('PALM SPRINGS ASOS, CA US2017-06-26'))) # pandas.core.series.Series
col_name_climate = fire_nearest_climate_df.columns
col_name_fire = sandiego_riverside_df.columns



full_column_names = np.append(col_name_climate, col_name_fire)

lst = []
for wi, wr in sandiego_riverside_df.iterrows():
    key = wr['stations'] + wr['started_date_str']
    
    if key in climate_dic:
        cl = climate_dic[key]
               
        lst.append(np.append(cl, wr.to_numpy()))

# all columns with climate and fire
climate_and_fire_full_df = pd.DataFrame(lst, columns = full_column_names)
climate_and_fire_full_df.head(1)

,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders,median_income,median_income_level,seaside_inland,stations,station_latitude,station_longitude,started_date_str
0,"PALM SPRINGS ASOS, CA US",33.8222,-116.5043,124.7,2017-06-26,9.17,0.0,116.0,85.0,320.0,330.0,31.1,38.9,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Riverside,6309,False,CAL FIRE/Riverside County Fire,<NA>,2017,True,/incidents/2017/6/26/manzanita-fire/,<NA>,<NA>,Riverside,33,<NA>,<NA>,<NA>,2018-01-09T11:08:00Z,<NA>,False,True,<NA>,<NA>,<NA>,33.530045,"Hwy 79 North, Lambs Canyon south of Dump Road,...",-116.594422,False,Manzanita Fire,100,<NA>,True,"The Manzanita Fire started on June 26, 2017 of...","Hwy 79 North, Lambs Canyon, Riverside County, ...",2017-06-26T15:10:00Z,Finalized,<NA>,<NA>,<NA>,<NA>,ca0c9a5f-7a1a-4903-b790-6eb3e85e7e64,2018-01-09T11:08:00Z,<NA>,63948.0,3,inland,"PALM SPRINGS ASOS, CA US",33.8222,-116.5043,2017-06-26


In [40]:
climate_and_fire_full_df.to_csv(r'D:\yeshiva\2 semester\math\final\data\California_climate_and_fire_full.csv', index=False)

In [41]:
lst = []
for wi, wr in sandiego_riverside_df.iterrows():
    # get the fire if it mach the station and the date
    lst.append(climate_daily_df[(climate_daily_df['NAME'] == wr['stations']) & (climate_daily_df['DATE'] == wr['started_date_str'])])

lst

[                          NAME  LATITUDE  LONGITUDE  ELEVATION        DATE  \
 6738  PALM SPRINGS ASOS, CA US   33.8222  -116.5043      124.7  2017-06-26   
 
       AWND  PRCP   TMAX  TMIN   WDF2   WDF5  WSF2  WSF5  WT01  WT02  WT03  \
 6738  9.17   0.0  116.0  85.0  320.0  330.0  31.1  38.9   NaN   NaN   NaN   
 
       WT04  WT05  WT07  WT08  WT10  WT11     County  
 6738   NaN   NaN   1.0   NaN   NaN   NaN  Riverside  ,
                                           NAME  LATITUDE  LONGITUDE  \
 340  CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US  33.12806 -117.27944   
 
      ELEVATION        DATE  AWND  PRCP  TMAX  TMIN  WDF2  WDF5  WSF2  WSF5  \
 340      100.0  2017-12-07  6.26   0.0  82.0  50.0  90.0  90.0  23.0  28.0   
 
      WT01  WT02  WT03  WT04  WT05  WT07  WT08  WT10  WT11     County  
 340   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  San Diego  ,
                                     NAME  LATITUDE  LONGITUDE  ELEVATION  \
 7901  RIVERSIDE MUNICIPAL AIRPORT, CA US  

In [42]:
len(lst)

164

In [43]:
# data of the wildfire start date and the nearest climate stations
fire_nearest_climate_df = pd.concat(lst)

In [44]:
fire_nearest_climate_df.head(2)

,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County
6738,"PALM SPRINGS ASOS, CA US",33.82220,-116.50430,124.7,2017-06-26,9.17,0.0,116.0,85.0,320.0,330.0,31.1,38.9,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Riverside
340,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-12-07,6.26,0.0,82.0,50.0,90.0,90.0,23.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego


In [45]:
no_fire_climate_new_df = climate_daily_df.drop(fire_nearest_climate_df.index)
no_fire_climate_new_df.head(2)

,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County
0,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-01-01,4.92,0.00,57.0,44.0,210.0,210.0,15.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
1,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-01-02,1.79,0.03,58.0,50.0,250.0,250.0,8.1,10.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego


In [46]:
no_fire_climate_new_df.shape

(9697, 23)

### 3.3) add flag for wildfire in 2017 - 2019 

In [47]:
# wildfire_df['median_income'] = wildfire_df.apply(lambda row: income_dict.get(row['Counties']), axis=1)
fire_nearest_climate_df['fire_flag'] = fire_nearest_climate_df.apply(lambda row: 1 if row['DATE'] > '1' else 0, axis=1)
fire_nearest_climate_df.head(2)

,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County,fire_flag
6738,"PALM SPRINGS ASOS, CA US",33.82220,-116.50430,124.7,2017-06-26,9.17,0.0,116.0,85.0,320.0,330.0,31.1,38.9,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Riverside,1
340,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-12-07,6.26,0.0,82.0,50.0,90.0,90.0,23.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,1


In [48]:
fire_nearest_climate_df.describe()

,LATITUDE,LONGITUDE,ELEVATION,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,fire_flag
count,164.000000,164.000000,164.000000,164.000000,163.000000,164.000000,161.000000,164.000000,162.000000,164.000000,162.000000,41.0,7.0,1.0,0.0,3.0,24.0,19.0,0.0,0.0,164.0
mean,33.527316,-117.066355,210.843293,5.998293,0.001472,90.542683,62.155280,262.926829,254.320988,18.041463,23.856173,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
std,0.508569,0.510017,96.805771,2.464238,0.018798,13.119250,8.743254,62.924393,71.790556,4.680109,6.362379,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0
min,32.572220,-117.438610,100.000000,2.460000,0.000000,61.000000,44.000000,20.000000,10.000000,8.900000,12.100000,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
25%,33.105420,-117.438610,124.700000,4.700000,0.000000,79.750000,57.000000,260.000000,252.500000,14.775000,19.000000,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
50%,33.822200,-117.279440,245.200000,5.370000,0.000000,92.000000,62.000000,270.000000,270.000000,17.000000,23.000000,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
75%,33.951940,-116.915830,245.200000,6.317500,0.000000,98.000000,67.000000,280.000000,280.000000,21.000000,27.100000,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
max,33.951940,-114.714200,424.600000,15.430000,0.240000,119.000000,91.000000,360.000000,360.000000,35.100000,46.100000,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0


In [49]:
fire_nearest_climate_df.to_csv(r'D:\yeshiva\2 semester\math\final\data\California_Fire_nearest_climate.csv', index=False)

In [50]:
# flag the no fire climate date
no_fire_climate_new_df['fire_flag'] = no_fire_climate_new_df.apply(lambda row: 0 if row['DATE'] > '1' else 1, axis=1)
no_fire_climate_new_df.head(2)

,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County,fire_flag
0,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-01-01,4.92,0.00,57.0,44.0,210.0,210.0,15.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,0
1,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",33.12806,-117.27944,100.0,2017-01-02,1.79,0.03,58.0,50.0,250.0,250.0,8.1,10.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,0


In [51]:
no_fire_climate_new_df.to_csv(r'D:\yeshiva\2 semester\math\final\data\California_No_Fire_nearest_climate.csv', index=False)

### 3.4) combine a sub set from fire and no fire set

In [52]:
no_fire_sample = no_fire_climate_new_df.sample(n=164)
no_fire_sample.head(2)

,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,County,fire_flag
9633,"BLYTHE ASOS, CA US",33.61860,-114.71420,120.4,2019-06-01,8.28,0.0,96.0,63.0,180.0,190.0,19.9,25.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,0
8570,"RIVERSIDE MUNICIPAL AIRPORT, CA US",33.95194,-117.43861,245.2,2019-07-03,5.82,0.0,87.0,60.0,270.0,250.0,19.9,25.1,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Riverside,0


In [53]:
no_fire_sample.describe()

,LATITUDE,LONGITUDE,ELEVATION,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT07,WT08,WT10,WT11,fire_flag
count,164.000000,164.000000,164.000000,164.000000,162.000000,164.000000,164.000000,164.000000,163.000000,164.000000,163.000000,44.0,8.0,0.0,0.0,0.0,10.0,23.0,0.0,0.0,164.0
mean,33.176095,-116.822418,142.528049,5.811524,0.031605,77.536585,57.975610,249.207317,247.177914,16.194512,21.162577,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0.0
std,0.488621,0.786442,118.754937,2.328053,0.158066,13.650947,10.474598,69.907781,72.661138,5.226375,6.362643,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0
min,32.572220,-117.438610,4.600000,1.340000,0.000000,55.000000,33.000000,10.000000,20.000000,6.900000,10.100000,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0.0
25%,32.733600,-117.200000,100.000000,4.415000,0.000000,67.750000,50.000000,230.000000,220.000000,13.000000,17.000000,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0.0
50%,33.037500,-117.139440,124.700000,5.370000,0.000000,74.000000,57.000000,270.000000,270.000000,15.000000,19.900000,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0.0
75%,33.618600,-116.915830,157.000000,6.930000,0.000000,83.250000,65.000000,290.000000,290.000000,18.100000,23.900000,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0.0
max,33.951940,-114.714200,424.600000,15.660000,1.200000,116.000000,93.000000,360.000000,360.000000,35.100000,42.100000,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0.0
